## 项目:travis data mining for tik tok vedio
- **任务**：一般来说，为了吸引观众的注意力，广告视频的长度、音频、文本位置和画面会有与众不同之处。我们将通过对抖音平台上视频的时长、声音频谱、视频光谱、文字分布和画面变化等特征，构建一套商用广告视频识别系统来快速区分出投稿视频中的商用广告。
- **步骤**：    
                        1. 数据的探索与问题分析；
                        2. 清洗数据；
                            a.处理缺失值；
                            b.标签转换；
                            c.查看重复；
                        3. 特征工程；
                            a.特征选择；
                            b.特征生成；
                            c.特征分箱；
                        4. 选择模型进行交叉验证和网格搜索；
                        5. 模型的集成；
                        6. 模型评价的深入思考；
                        7. 进一步思考该项目。                     

>**提示：**Code 和 Markdown 区域可通过 **Shift + Enter** 快捷键运行。此外，Markdown可以通过双击进入编辑模式。

---
## 步骤一：数据的探索与问题分析

阅读《基于机器学习的商用广告视频识别.pdf》文档，明确分析任务。
阅读《数据集和变量说明.pdf》文档，简单了解数据，并描述数据基本特征。
数据文件如下：
- commercial_vedio_data.csv

In [ ]:
# 导入依赖库
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
#加载数据
data = pd.read_csv('commercial_vedio_data.csv',index_col=0)
col_name= data.columns[:-2]
label_name = data.columns[-1]
#查看数据的标签
print('训练集的标签：{}\n'.format(label_name))
#查看数据的特征
print('训练集的特征：{}\n'.format(col_name))
#查看数据的shape
print('训练集的形状：{}'.format(data.shape))

### 举个例子
打印数据集的前5行数据

In [ ]:
# TODO：打印data的前五行数据

### 举个例子
查看data数据集中标签('lables')的分布

In [ ]:
#data['labels']

In [1]:
# TODO：画出标签分布的直方图
#frequency = dict()
#for num in data['labels']:
  #  frequency[num] = frequency.get(num, 0) + 1
#plt.bar(list(frequency.keys()),
        #list(frequency.values()))
#plt.show()
#以上是travis的示例
#你自己写一个其他版本

NameError: name '以上是travis的示例' is not defined

### 举个例子
查看data数据集特征的的分布，对于describe善用转置可展示更多特征<br>
真正了解你的数据，而非走形式

In [ ]:
# TODO：描述数据中特征的分布(表1)
data.describe()

In [ ]:
# TODO：描述数据
fig=plt.figure(figsize=(6,6))
ax1=fig.add_subplot(1,1,1)
ax1.hist(data['1'],bins=2000,alpha=0.7)
plt.title('times')
plt.xlim([0,1000])
plt.show()

In [ ]:
# TODO：描述数据中特征的分布(表3)
#fig=plt.figure(figsize=(7,7))
#ax1=fig.add_subplot(1,1,1)
#ax1.hist(data['4124'],bins=2000,color='g',alpha=0.65)
#plt.title('4124 feature')
#plt.xlim([0,1])
#plt.show()

#以上是travis示例 自己写一个其他的对比 你的可能更好

---
## 步骤二：清洗数据

在这一步中将开始清洗数据

通过观察数据我们发现：

- 数据需要查看是否存在缺失值；
- 数据需要查看是否存在重复样本；
- 虽然不同样本的相同特征取值变化不大，但特征间数值差距大；
- 数据的标签需要变化。


统计数据中的缺失值的数量
- 统计数据集有多少行有缺失
- 统计数据集有多少列有缺失

In [ ]:
# 统计数据集中有多少行、列存在缺失
#提示 用data.isnull().any()

### 举个例子
统计数据中的重复样本的数量
- 统计数据集有多少行重复

In [ ]:
# TODO：统计数据集中有多少行重复

### 注意
从上面我们可以看出数据的缺失值很多。结合文档中介绍的我们得到这些数据的方式，你觉得应如何处理缺失值？<br>


下面给出一种思路。

In [ ]:
# 将所有缺失值填补
#data = data.fillna(data.mean())
#也可以用0填充 数据很特殊

### 注意
感觉特征间的尺度差距较大，注意在建模前结合模型要求进行适当的处理。

### 标签数据转换
将labels的值中的“-1”转换为0

In [ ]:
# 将标签中的‘-1’转换为‘0’

### 分类特征重编码

从上面的**数据探索**中的表中，可以看到有几个特征是无序的分类特征。因为无序特征各属性之间不能比较大小，通常情况下，要求无序特征（称为无序类别变量）被转换。无序转换类别变量的一种流行的方法是使用**独热编码**方案。独热编码为每一个无序分类特征的每一个可能的类别创建一个_“虚拟”_变量。例如，假设`someFeature`有三个可能的取值`A`，`B`或者`C`。我们将把这个特征编码成`someFeature_A`, `someFeature_B`和`someFeature_C`.

| someFeature | someFeature_A | someFeature_B | someFeature_C |
| :-: | :-: | :-: | :-: |
|  A  | 1 | 0 | 0 |
|  B  | 0 | 1 | 0 |
|  C  | 0 | 0 | 1 |

 - 使用[`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies)对`'features_raw'`数据来施加一个独热编码。


In [ ]:
# 是否需要对特征进行one-hot编码
#要继续改
index = train_features_raw.shape[0]
temp = pd.get_dummies(pd.concat([train_features_raw,test_features_raw],axis=0))
train = temp.iloc[:index]
test = temp.iloc[index:]

## 步骤三：特征工程

- 特征过滤
- 特征生成
- 特征分箱

可综合多种方法进行特征工程。一是使用某些方法生成新的特征纳入模型进行预测；
二是通过某些方法进行特征过滤，减少纳入模型的特征数量；
三是对连续特征进行特征分箱，离散特征进行特征组合。

## 分离特征和标签

In [ ]:
# 分离特征和标签
train_label_raw = train['labels']
train_features_raw = train.drop('labels', axis=1)
print(train_features_raw)

### 划分训练集和测试集
为避免测试集在特征工程步骤中被引入训练集信息，在进行特征工程前划分训练集和验证集。

In [ ]:
# 将数据集划分为训练集和测试集
trains, tests = train_test_split(data, test_size=0.75)

### 使用随机森林进行特征选择
可用其他方法进行特征选择

先来个示例:
`'feature_importance_'` 属性的scikit学习分类器（例如随机森林）。         
`'feature_importance_'` 属性是对特征的重要性排序的函数。
在之后的步骤中将使用这个分类器拟合训练集数据并使用这个属性来对特征的重要程度进行排序。

In [ ]:
#使用随机森林进行特征选择，首先在训练集拟合随机森林模型


In [ ]:
#将特征的重要性程度进行排序
#看了数据集就会清楚下,后面40多-110行之间有不少的缺失


In [ ]:
# 保留训练集和测试集中对于分类最重要的前n个特征（自主考虑保留多少个特征?）
# 以保留对于分类最重要的五十个特征为例，具体多少我也不知道哈哈
#imp = np.argsort(rf.feature_importances_)[::-1]
#imp_slct = imp[:50]

In [ ]:
X_train_slct=pd.DataFrame(X_train).iloc[:,imp_slct]
X_test_slct=X_test.iloc[:,imp_slct]
#请生成一个特征排序的热力图，读入特征排序后数组里的前五十个特征

###  使用 PCA进行特征生成

PCA除了用于降维外，还可用于特征生成，即将选择出的主成分与原数据合并，一般会提升原数据的预测能力。

In [ ]:
# 对训练集使用L1PCA生成新特征,首先查看各主成分解释方差比例


In [ ]:
#对训练集使用PCA生成新特征,根据累计贡献率，保留前5个主成分
pca1 = PCA(6)
pc = pd.DataFrame(pca1.fit_transform(X_train))
pc.index =  X_train.index
X_train_pca = X_train.join(pc)

In [ ]:
# 对测试集进行相同的操作，注意测试集上直接使用pca中的transform函数



###  使用决策树进行特征分箱

可以用所知道的其他方法进行此步

特征分箱是常用的一种特征工程方法，它将连续变量离散化。合理的离散化将提升数据的预测能力。
实现特征分箱的方法很多：
- 简单地将数据等距分成n份
- 简单地将数据等频分成n份
- 根据卡方统计量优化分箱
- 根据决策树优化分箱
- 根据其他算法优化分箱

一般还需根据WOE和IV评价分箱效果，在此不做进一步拓展了。
以下代码是两函数是决策树优化分箱的示例(copy)
- 注意:可以尝试将他们改装成类，实现更方便的调用?

In [ ]:
# 将训练集及其标签、测试集及其标签合并，以方便函数使用


In [ ]:
'''
cut_bin参数说明
    df：训练集名称，如train
    label：标签名称，如'labels'
    max_depth：决策树最大深度
    p：最小叶子节点数与样本量的比例
cut_bin输出说明
    df_bin：分箱后的数据集
    dict_bin：储存分箱参数的字典，包含了用以评价分箱的WOE和IV
'''
'''
cut_test_bin参数说明
    df：测试集名称，如test
    label：标签名称，如'labels'
    train_dict_bin：训练集分箱生成的储存参数的字典
cut_test_bin输出说明
    df_bin：分箱后的数据集
    dict_bin：储存分箱参数的字典，包含了用以评价分箱的WOE和IV
    '''

In [ ]:
#  cut_bin对训练集进行分箱
#  cut_test_bin对测试集进行分箱
def cut_bin(df,label,max_depth,p):
    df_bin = df[[label]]
    df_feature = df.drop([label],axis=1)
    dict_bin = {}
    for col in df_feature.columns:
        get_model = DecisionTreeClassifier(max_depth=max_depth,min_samples_leaf=int(p*len(df)))
        get_cut_point = get_model.fit(df[col].values.reshape(-1,1),df[label].values.reshape(-1,1))
        cut_point = get_cut_point.tree_.threshold[get_cut_point.tree_.threshold!=-2]
        
        N_split = np.zeros_like(df[col])
        inter_range = []
        if len(cut_point)==1:
            N_split[np.array(df[col]<cut_point[0])]=1
            N_split[np.array(df[col]>=cut_point[0])]=2
            inter_range=[[1,-100000000,cut_point[0]],[2,cut_point[0],100000000]]
        elif len(cut_point)>1:
            cut_point.sort()
            N_split[np.array(df[col]<cut_point[0])]=1
            inter_range=[[1,-100000000,cut_point[0]]]
            for i in range(len(cut_point)-1):
                N_split[np.array((df[col]>=cut_point[i]) & (df[col]<cut_point[i+1]))]=i+2
                inter_range=inter_range+[[i+2,cut_point[i],cut_point[i+1]]]
            N_split[np.array(df[col]>=cut_point[len(cut_point)-1])]=len(cut_point)+1
            inter_range=inter_range+[[len(cut_point)+1,cut_point[len(cut_point)-1],100000000]]
        else:
            N_split=1
            inter_range=np.array([1,-100000000,100000000]).reshape(1,-1)
        df_bin[col] = N_split
        inter_df = pd.DataFrame(inter_range)
        inter_df.columns=['bin','lower','upper']
        crosstable = pd.crosstab(df_bin[col],df_bin[label])
        crosstable.columns = ['notCommercial','Commercial']
        crosstable['all'] = crosstable['notCommercial']+crosstable['Commercial']
        crosstable['percent'] = crosstable['all']/sum(crosstable['all'])
        crosstable['c_rate'] = crosstable['Commercial']/crosstable['all']
        inter_df = pd.merge(inter_df, crosstable, left_on='bin', right_index=True)
        dict_bin[col] = inter_df
    return df_bin, dict_bin

def cut_test_bin(df, label, train_dict_bin):
    df_bin = df[[label]]
    df_feature = df.drop([label],axis=1)
    dict_bin = {}
    for col in df_feature.columns:
        train_bin = train_dict_bin[col]
        splited = pd.Series([np.nan]*len(df[col]))
        for i in range(len(train_bin['bin'])):
            splited[((df[col]>=train_bin['lower'][i]) & (df[col]<train_bin['upper'][i])).tolist()]=train_bin['bin'][i]
            df_bin[col]=splited.tolist()
        crosstable = pd.crosstab(df_bin[col],df_bin[label])
        crosstable.columns = ['notCommercial','Commercial']
        crosstable['all'] = crosstable['notCommercial']+crosstable['Commercial']
        crosstable['percent'] = crosstable['all']/sum(crosstable['all'])
        crosstable['c_rate'] = crosstable['Commercial']/crosstable['all']
        inter_df = pd.merge(train_bin[['bin','lower','upper']], crosstable, left_on='bin', right_index=True, how='left')
        dict_bin[col] = inter_df
    return df_bin, dict_bin   

In [ ]:
from sklearn.tree import DecisionTreeClassifier
#调用函数完成特征分箱

### 分重新离特征和标签

In [ ]:
#再次分离特征和标签，训练集和测试集都要进行,之前我忘过


## 步骤四：选择模型进行交叉验证和网格搜索


选择合适的模型以及评估方式，使用交叉验证和网格搜索建立模型，并选择合适的参数,打印出交叉验证的结果。
- clf_model是分类模型。
- 注意:模型对数据规整的需求。

#模型选择、交叉验证、网格搜索
clf_model = None

# 这里演示最简单的模型更多的模型选择和参数调整

# 对比使用随机森林和(xgboost/lightgbm二选一)
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_features=16,max_depth=12,n_estimators=2048,n_jobs=-1,random_state=0)
rf.fit(X_train_bin,y_train)

### 思考

- 对于分类预测稳定性问题，使用accuracy、混淆矩阵和AUC指标来评估模型。

- 注意： 除此以外，我觉得还可以设计一个随机猜测函数作为baseline来进一步查看模型相对于随机猜测是否有很大的优?


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, accuracy_score
# 分类模型测试集上效果
# auc和混淆矩阵评估
y_train_pred_clf = rf.predict_proba(X_train_bin)
y_train_pred = rf.predict(X_train_bin)
y_test_pred_clf = rf.predict_proba(X_test_bin)
y_test_pred = rf.predict(X_test_bin)
# 评估训练集效果，直观判断是否过拟合
print('分类模型训练集表现：')
print('ml train model auc score {:.6f}'.format(roc_auc_score(y_train,y_train_pred_clf[:,1])))
print('---------------')
print('ml train model accuracy score {:.6f}'.format(accuracy_score(y_train,y_train_pred)))
print('---------------')
threshold = 0.5
print(confusion_matrix(y_train,(y_train_pred_clf>threshold)[:,1]))
# 评估测试集效果
print('分类模型测试集表现：')
print('ml model auc score {:.6f}'.format(roc_auc_score(y_test,y_test_pred_clf[:,1])))
print('---------------')
print('ml model accuracy score {:.6f}'.format(accuracy_score(y_test,y_test_pred)))
print('---------------')
threshold = 0.5
print(confusion_matrix(y_test,(y_test_pred_clf>threshold)[:,1]))
# 随机猜测函数对比
y_test_random_clf = np.random.uniform(low=0.0,high=1.0,size=len(y_test))

print('random model auc score {:.6f}'.format(roc_auc_score(y_test,y_test_random_clf)))
print('---------------')
print(confusion_matrix(y_test,(y_test_random_clf<=threshold).astype('int')))

## 步骤五：模型集成
尝试用stacking等方法对步骤四生成的不同学习器进行集成。

注意评价集成后的模型。

In [ ]:
##模型集成举例
ensemble_model_clf = None

## 步骤六：模型评价和思考
找到你认为最好的模型后，设定不同的分类阈值(threshold)仍会导致模型的实际应用效果有所不同，这无疑会影响模型的实际使用效果。ROC曲线是以模型在不同阈值下的真阳性率为纵轴，假阳性率为横轴绘制而成，它不仅可以不依靠阈值评价模型的预测效果。

In [ ]:
## 计算各阈值下假阳性率、真阳性率和AUC 
from sklearn.metrics import roc_curve, auc
fpr,tpr,threshold = roc_curve(y_test,y_test_pred_clf[:,1])
roc_auc = auc(fpr,tpr)


In [ ]:
## 假阳性率为横坐标，真阳性率为纵坐标做曲线


In [ ]:
##生成一个混淆矩阵看看！

使用ACC和AUC评价模型，但是针对找出商用视频这一目的，是否有其他更合适的指标？结合混淆矩阵说明。

## 创新点:
也可以将整个流程调整为更合理的方式；

尝试一下综合应用特征选择和特征生成，选择最适合本问题的方法；

可以在特征选择，**之后**再生成新的特征。特征选择可以采取包裹法、过滤法或其他嵌入法进行。

In [ ]:
##特征选择+特征生成